### Summation Notation

Linear programs typically involve sums over many decision variables. We introduce sigma notation ($\sum$) as a shorthand way to represent arbitrarily large sums.

Consider the set $I = \{1, 2, 3\}$. The expression 
\begin{equation}
\sum_{i \in I} a_i x_i
\end{equation}
is shorthand for the sum 
\begin{equation}
a_1 x_1 + a_2 x_2 + a_3 x_3.
\end{equation}
The symbol $\in$ is read as "in". 

What we gain from adopting this notation is that now if we consider $I = \{1, 2, \ldots, n\}$ for an arbitrary $n$,  $\sum_{i \in I} a_i x_i = a_1 x_1 + a_2 x_2 + \cdots + a_n x_n$. That is, we can write arbitrarily-sized sums in one shot.

### Knapsack problem

Suppose you are given a set of items $I = \{1, \ldots, n\}$, each with a value $v_i$ and a weight $w_i$. We have a knapsack that has a weight capacity of $b$. What is the highest valued collection of items that can go into the knapsack? Assume that you can put fractions of items into the knapsack.

Let $x_i, i \in I$ be the fraction of item $i$ that goes in the knapsack. We can use sigma notation to write expressions for the value and weight of our collection of items. The total value is given by $\sum_{i \in I} v_i x_i$ and the total weight is given by $\sum_{i \in I} w_i x_i$. Then we can formulate our problem as a linear program:

\begin{eqnarray}
\max_x && \sum_{i \in I} v_i x_i \\
\mbox{s.t.} && \sum_{i \in I} w_i x_i \le b \\
&& 0 \le x_i \le 1,\;\;i \in I.
\end{eqnarray}

Consider the following data:

In [ ]:
weights = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
values = [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240]
I = range(len(values))
capacity = 750

First, we're going to need to instantiate the decision variables and save references to them in a data structure. We'll use a Python list for this purpose.

In [ ]:
from gurobipy import *

In [ ]:
m = Model()

In [ ]:
item_selected = []
for i in I:
    item_selected.append(m.addVar(ub=1, obj=values[i], name='item_selected.' + str(i)))

In [ ]:
m.update()

Now we need to write the knapsack capacity constraint, which means we'll need an expression for the total weight of our collection of items (in terms of the decision variables we just created). We use a LinExpr object for this purpose, and we've got a few options to build up this object. The first uses some syntactic sugar that is only available through the Python API, namely the quicksum method.

In [ ]:
quicksum(weights[i]*item_selected[i] for i in I)

A slightly less slick way to acheive this is through a for loop.

In [ ]:
total_weight = LinExpr()
for i in I:
    total_weight += weights[i]*item_selected[i]
print total_weight

Here, the += operator is overloaded to return a LinExpr object as opposed to doing arithmetic. Operator overloading isn't available in Java, and is not advisable in .NET. Those API's provide methods on the LinExpr (really GRBLinExpr) object that do the same job.

In [ ]:
total_weight = LinExpr()
for i in I:
    total_weight.add(item_selected[i], weights[i])
print total_weight

However we build up the summation of the weights, we can use the LinExpr object as the left-hand side of a constraint.

In [ ]:
weight_con = m.addConstr(total_weight <= capacity, name='total_weight')

In [ ]:
m.update()

In [ ]:
m.write('knapsack.lp')

In [ ]:
!more knapsack.lp

In [ ]:
m.optimize()

Remember that Gurobi will minimize by default...

In [ ]:
m.ModelSense = GRB.MAXIMIZE

In [ ]:
m.optimize()

In [ ]:
for var in item_selected:
    print var.VarName, var.X

In [ ]:
for var in item_selected:
    var.vtype = GRB.BINARY

In [ ]:
m.optimize()

In [ ]:
for var in item_selected:
    print var.VarName, var.X

We can wrap this up in a method that takes in the data and returns a Model.

In [ ]:
def get_knapsack_model(capacity, weights, values):
    items = range(len(weights))
    m = Model()
    m.ModelSense = GRB.MAXIMIZE
    item_selected = [m.addVar(ub=1, obj=values[item], name="item_selected." + str(item))
                              for item in items]
    m.update()
    m.addConstr(quicksum(weights[item]*item_selected[item] for item in items) <= capacity)
    m.update()
    return m

Let's try this with a different set of inputs:

In [ ]:
m = get_knapsack_model(capacity=104,
                       weights=[25, 35, 45, 5, 25, 3, 2, 2],
                       values=[350, 400, 450, 20, 70, 8, 5, 5])

In [ ]:
m.optimize()

In [ ]:
for var in m.getVars():
    print var.VarName, var.X

We can see that we selected the first two items and 97.7% of the third. What if we aren't allowed to include fractions of items in the knapsack?

In [ ]:
for var in m.getVars():
    var.vtype = GRB.BINARY

In [ ]:
m.optimize()

In [ ]:
for var in m.getVars():
    print var.VarName, var.X

Now we've really solved an integer program, and it's worth noting that the solution is considerably different than that of the linear programming version of the problem.